In [1]:
!pip install --upgrade pip
!pip install keras
!pip install --upgrade jupyter ipywidgets
!pip install --upgrade tensorflow
!pip install --upgrade tf-keras tensorflow-text
!pip install keras-core
!pip install --upgrade tensorflow-hub



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.3.2
    Uninstalling pip-23.3.2:
      Successfully uninstalled pip-23.3.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.0/215.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 38.2 MB/s eta 0:00:00
  Attempting uninstall: widgetsnbextension
    Found existing installation: widgetsnbextension 3.6.6
    Uninstalling widgetsnbextension-3.6.6:
      Successfully uninstalled widgetsnbextension-3.6.6
  Attempting uninstall: jupyterlab-widgets
    Found existing installation: jupyterlab-widgets 3.0.9
    Uninstalling jupyterlab-widgets-3.0.9:
      Successfully uninstalled jupyterlab-widgets-3.0.9
  Attempting uninstall: ipywidgets
    Found existing installation: ipywidgets 7.7.1
    Uninstalling ipywidgets-7.7.1:
      Successfully unin

In [2]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'
import re
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from kaggle_datasets import KaggleDatasets

import tensorflow as tf

import tf_keras as keras

from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess_input
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [3]:
from tensorflow.keras.applications.efficientnet import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess_input

In [4]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [5]:
print(os.getenv('TF_USE_LEGACY_KERAS'))

1


In [6]:
# import tensorflow as tf

# try:
#   # Attempt TPU detection using `tf.distribute.cluster_resolver.TPUClusterResolver`
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='local')
#   print('Running on TPU:', tpu.cluster_spec().as_dict().get('worker', 'CPU/GPU') if tpu else 'CPU/GPU')

# except ValueError:
#   # Handle the case where no TPU is found
#   print('ERROR: Not connected to a TPU runtime!')
#   raise BaseException('No TPU detected. Consider using CPU or GPU for training.')  # More informative error message

# # Regardless of TPU availability, configure the strategy
strategy = tf.distribute.experimental.CentralStorageStrategy()  # Efficient strategy for most cases

In [7]:
# Directly use the Kaggle dataset path without get_gcs_path
GCS_PATH = '/kaggle/input/cassava-leaf-disease-classification'

In [8]:
tfrec_fnames = tf.io.gfile.glob(GCS_PATH + '/train_tfrecords/ld_train*.tfrec')
len(tfrec_fnames)

16

In [9]:
label_to_disease = pd.read_json(os.path.join(GCS_PATH, 'label_num_to_disease_map.json'), typ='series')

In [10]:
train_csv = pd.read_csv(os.path.join(GCS_PATH, 'train.csv'))

In [11]:
train_csv['disease'] = train_csv['label'].map(label_to_disease)
train_csv.head()

,image_id,label,disease
0,1000015157.jpg,0,Cassava Bacterial Blight (CBB)
1,1000201771.jpg,3,Cassava Mosaic Disease (CMD)
2,100042118.jpg,1,Cassava Brown Streak Disease (CBSD)
3,1000723321.jpg,1,Cassava Brown Streak Disease (CBSD)
4,1000812911.jpg,3,Cassava Mosaic Disease (CMD)


In [12]:
# 75:25 train:valid
train_fnames = tfrec_fnames[:12]
valid_fnames = tfrec_fnames[12:]
print(len(train_fnames), len(valid_fnames))

12 4


In [13]:
early_stop = keras.callbacks.EarlyStopping(monitor = 'val_loss', min_delta = 0.001, 
                                              patience = 5, mode = 'min', verbose = 1,
                                              restore_best_weights = True)

reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor = 'val_loss', factor = 0.3, 
                                                 patience = 2, min_delta = 0.001, 
                                                 mode = 'min', verbose = 1)

In [14]:
AUTO = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
IMAGE_SIZE = [512, 512]
print(strategy.num_replicas_in_sync)

1


In [15]:
def _parse_function(proto):
    # feature_description needs to be defined since datasets use graph-execution
    # - its used to build their shape and type signature
    feature_description = {
        'image': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'image_name': tf.io.FixedLenFeature([], tf.string, default_value=''),
        'target': tf.io.FixedLenFeature([], tf.int64, default_value=-1)
    }

    parsed_features = tf.io.parse_single_example(proto, feature_description)
    image = tf.image.decode_jpeg(parsed_features['image'], channels=3)
    image = tf.cast(image, tf.float32) # :: [0.0, 255.0]
    image = tf.reshape(image, [*IMAGE_SIZE, 3])
    target = tf.one_hot(parsed_features['target'], depth=5)
    return image, target

In [16]:
def load_dataset(tfrecords_fnames):
    raw_ds = tf.data.TFRecordDataset(tfrecords_fnames, num_parallel_reads=AUTO)
    parsed_ds = raw_ds.map(_parse_function, num_parallel_calls=AUTO)
    return parsed_ds

In [17]:
def build_train_ds(train_fnames, with_aug=False):
    ds = load_dataset(train_fnames)

    def data_augment(image, target):
        modified = tf.image.random_flip_left_right(image)
        modified = tf.image.random_flip_up_down(image)
        #modified = tf.image.random_brightness(modified, 0.2)
        #modified = tf.image.random_contrast(modified, 0.2, 0.5)
        #modified = tf.image.random_hue(modified, 0.2)
        #modified = tf.image.random_saturation(modified, 5, 10)
        modified = tf.clip_by_value(modified, 0.0, 255.0)
        return modified, target

    if with_aug:
        ds = ds.map(data_augment, num_parallel_calls=AUTO)

    return ds.repeat().shuffle(2048).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)

In [18]:
def build_valid_ds(valid_fnames):
    ds = load_dataset(valid_fnames)
    ds = ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)
    return ds

In [19]:
def count_data_items(filenames):
    n = [int(re.compile(r"-([0-9]*)\.").search(fname).group(1)) for fname in filenames]
    return np.sum(n)

n_train = count_data_items(train_fnames)
n_valid = count_data_items(valid_fnames)
train_steps = count_data_items(train_fnames) // BATCH_SIZE
print("TRAINING IMAGES:", n_train, ", STEPS PER EPOCH:", train_steps)
print("VALIDATION IMAGES:", n_valid)

TRAINING IMAGES: 16045 , STEPS PER EPOCH: 501
VALIDATION IMAGES: 5352


In [20]:
def preprocess_fn(image, label):
    image = image / 255.0
    image = tf.image.resize(image, (224, 224))
    label = tf.concat([label, [0]], axis=0)
    return image, label

In [21]:
train_fnames = tfrec_fnames[:12]
valid_fnames = tfrec_fnames[12:]

train_ds = load_dataset(train_fnames)
train_ds = train_ds.map(preprocess_fn, num_parallel_calls=AUTO)
train_ds = train_ds.repeat().shuffle(2048).batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)

valid_ds = load_dataset(valid_fnames)
valid_ds = valid_ds.map(preprocess_fn, num_parallel_calls=AUTO)
valid_ds = valid_ds.batch(BATCH_SIZE, drop_remainder=True).prefetch(AUTO)

train_steps = count_data_items(train_fnames) // BATCH_SIZE

In [22]:
img, label = next(iter(train_ds))
print(img.numpy().max(), img.shape, img.dtype)

1.0 (32, 224, 224, 3) <dtype: 'float32'>


In [23]:
os.environ["TFHUB_CACHE_DIR"] = "/kaggle/working"
with strategy.scope():
    load_locally = tf.saved_model.LoadOptions(experimental_io_device='/job:localhost')
    cassava = hub.KerasLayer('https://tfhub.dev/google/cropnet/classifier/cassava_disease_V1/2', trainable=True, load_options=load_locally)
    model = keras.Sequential([keras.Input(shape=(224,224,3)),
                                 cassava])
        

In [24]:
#model.load_weights("/kaggle/working/preTrainedCropnet.keras")

In [25]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-5),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [26]:
model.fit(train_ds, validation_data=valid_ds,
          epochs=500, steps_per_epoch=train_steps,
          callbacks=[reduce_lr, early_stop])

Epoch 1/500


I0000 00:00:1714775110.857484     106 service.cc:145] XLA service 0x7fdd0a518220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1714775110.857542     106 service.cc:153]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1714775111.004503     106 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


501/501 [==============================] - 126s 114ms/step - loss: 2.1299 - accuracy: 0.7294 - val_loss: 0.4883 - val_accuracy: 0.8767 - lr: 1.0000e-05
Epoch 2/500
501/501 [==============================] - 52s 104ms/step - loss: 0.4593 - accuracy: 0.8633 - val_loss: 0.3422 - val_accuracy: 0.8911 - lr: 1.0000e-05
Epoch 3/500
501/501 [==============================] - 52s 104ms/step - loss: 0.3726 - accuracy: 0.8804 - val_loss: 0.3275 - val_accuracy: 0.8913 - lr: 1.0000e-05
Epoch 4/500
501/501 [==============================] - 51s 102ms/step - loss: 0.3416 - accuracy: 0.8868 - val_loss: 0.3098 - val_accuracy: 0.8971 - lr: 1.0000e-05
Epoch 5/500
501/501 [==============================] - 52s 103ms/step - loss: 0.3099 - accuracy: 0.8973 - val_loss: 0.2983 - val_accuracy: 0.8999 - lr: 1.0000e-05
Epoch 6/500
501/501 [==============================] - 52s 104ms/step - loss: 0.2978 - accuracy: 0.8981 - val_loss: 0.2896 - val_accuracy: 0.8999 - lr: 1.0000e-05
Epoch 7/500
501/501 [============

In [27]:
model.save("preTrainedCropnet5.keras")

In [28]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 6)                 4234118   
                                                                 
Total params: 4234118 (16.15 MB)
Trainable params: 4209718 (16.06 MB)
Non-trainable params: 24400 (95.31 KB)
_________________________________________________________________
